# Fictitious Names

### Introduction:

This time you will create a data again 

Special thanks to [Chris Albon](http://chrisalbon.com/) for sharing the dataset and materials.
All the credits to this exercise belongs to him.  

In order to understand about it go [here](https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/).

### Step 1. Import the necessary libraries

In [4]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T


# Generando session

spark = SparkSession.builder\
                     .appName('fakeNames')\
                     .getOrCreate()
                     
                     

### Step 2. Create the 3 DataFrames based on the following raw data

In [35]:
raw_data_1 = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}

raw_data_2 = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}

raw_data_3 = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_id': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}

### Step 3. Assign each to a variable called data1, data2, data3

In [7]:
df1 = pd.DataFrame(raw_data_1)
df2 = pd.DataFrame(raw_data_2)
df3 = pd.DataFrame(raw_data_3)

In [ ]:
data1 = spark.createDataFrame(df1)
data2 = spark.createDataFrame(df2)
data3 = spark.createDataFrame(df3)


In [48]:
data = []
for i, _ in enumerate(raw_data_1.items()):
    
    combinado = [raw_data_1[key][i] for key in raw_data_1.keys()]
    
    data.append(tuple(combinado))


data

[('1', 'Alex', 'Anderson'), ('2', 'Amy', 'Ackerman'), ('3', 'Allen', 'Ali')]

In [51]:
schema = T.StructType([
    T.StructField('subject_id', T.StringType()),
    T.StructField('first_name', T.StringType()),
    T.StructField('last_name', T.StringType()),

])

In [52]:
otra_forma = spark.createDataFrame(data, schema=schema)

In [53]:
otra_forma.show()

+----------+----------+---------+
|subject_id|first_name|last_name|
+----------+----------+---------+
|         1|      Alex| Anderson|
|         2|       Amy| Ackerman|
|         3|     Allen|      Ali|
+----------+----------+---------+



### Step 4. Join the two dataframes along rows and assign all_data

### Step 5. Join the two dataframes along columns and assing to all_data_col

### Step 6. Print data3

### Step 7. Merge all_data and data3 along the subject_id value

### Step 8. Merge only the data that has the same 'subject_id' on both data1 and data2

### Step 9. Merge all values in data1 and data2, with matching records from both sides where available.